In [ ]:
import numpy as np
import time as time
import os
import glob
import constants

# Set up matplotlib and use a nicer set of plot parameters
%config InlineBackend.rc = {}
import matplotlib as mpl
mpl.rc('mathtext',fontset='stixsans')
mpl.rc('figure', facecolor="white")
#matplotlib.rc_file("../../templates/matplotlibrc")
import matplotlib.pyplot as plt
#import matplotlib.colors as colors
# %matplotlib notebook

from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord

# Define cosmology
cosmo = constants.COSMOLOGY

PiBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins23_pi_0-30hMpc.txt')
SigBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins10_sigma_0-30hMpc.txt')

PiBins0 = ascii.read(PiBin_fil)
SigBins0 = ascii.read(SigBin_fil)

PiEdges = PiBins0['pi_edges'].data
SigEdges = SigBins0['sigma_edges'].data

# Convert bin boundaries from Mpc/h to Mpc
PiEdges  = PiEdges/(len(PiEdges)*[cosmo.h])
SigEdges = SigEdges/(len(SigEdges)*[cosmo.h])

print('Pi bin edges in Mpc:')
print(PiEdges)
print('Sigma bin edges in Mpc:')
print(SigEdges)

GAL_DIR = os.path.join(constants.XCORR_DIR_BASE, 'mock', 'gal')
MOCK_XCORR_DIR = os.path.join(constants.XCORR_DIR_BASE, 'mock', 'crosscorr')

gnum = 1
absnum= 64

PiBound = (min(PiEdges), max(PiEdges) )

# Read in galaxies to provide counts in plot titles
galfil = os.path.join(GAL_DIR, 'cat_galmock_nonuniq_{0:03d}.dat'.format(gnum))
gal = ascii.read(galfil,format='ipac')
#print(gal.columns)

gal_3d = gal[gal['source'] == '3DHST']
gal_zD = gal[gal['source'] == 'zDeep']
gal_mosdef = gal[gal['source'] == 'MOSDEF']
gal_vuds = gal[gal['source']=='VUDS']
gal_clamato=gal[gal['source']=='CLAMATO']

In [ ]:

XCorr_3d=np.load(os.path.join(MOCK_XCORR_DIR, "xcorrmock_3dhst_g{0:03d}_a{1:03d}_{2}.npy".format(gnum,absnum, constants.DATA_VERSION)))
XCorrPlot_3d = np.transpose(XCorr_3d)
    
XCorr_zD = np.load(os.path.join(MOCK_XCORR_DIR, "xcorrmock_zDeep_g{0:03d}_a{1:03d}_{2}.npy".format(gnum,absnum, constants.DATA_VERSION)))
XCorrPlot_zD = np.transpose(XCorr_zD)
    
XCorr_mosdef = np.load(os.path.join(MOCK_XCORR_DIR, "xcorrmock_mosdef_g{0:03d}_a{1:03d}_{2}.npy".format(gnum,absnum, constants.DATA_VERSION)))
XCorrPlot_mosdef = np.transpose(XCorr_mosdef)
    
XCorr_vuds = np.load(os.path.join(MOCK_XCORR_DIR, "xcorrmock_vuds_g{0:03d}_a{1:03d}_{2}.npy".format(gnum,absnum, constants.DATA_VERSION)))
XCorrPlot_vuds = np.transpose(XCorr_vuds)

XCorr_clamato = np.load(os.path.join(MOCK_XCORR_DIR, "xcorrmock_clamato_g{0:03d}_a{1:03d}_{2}.npy".format(gnum,absnum, constants.DATA_VERSION)))
XCorrPlot_clamato = np.transpose(XCorr_clamato)


In [ ]:
X, Y = np.meshgrid(SigEdges, PiEdges)

SigMax = 35.
PiMin = -40.
PiMax = 40.

fig, ((ax1, ax2, ax3),(ax4,ax5,ax6))= plt.subplots(2,3, figsize=(12,14))

print(len(X), len(Y), np.shape(XCorrPlot_mosdef))

ax1.pcolormesh(X, Y, XCorrPlot_mosdef,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax1.set_aspect('equal')
ax1.set_xlim(np.min(X), SigMax)
ax1.set_ylim(PiMin, PiMax)
ax1.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax1.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax1.set_title("MOSDEF: %i galaxies" % len(gal_mosdef))

ax2.pcolormesh(X, Y, XCorrPlot_vuds,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax2.set_aspect('equal')
ax2.set_xlim(np.min(X), SigMax)
ax2.set_ylim(PiMin, PiMax)
ax2.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax2.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax2.set_title("VUDS: %i galaxies" % len(gal_vuds))

ax3.pcolormesh(X, Y, XCorrPlot_zD,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax3.set_aspect('equal')
ax3.set_xlim(np.min(X), SigMax)
ax3.set_ylim(PiMin, PiMax)
ax3.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax3.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax3.set_title("zCOSMOS: %i galaxies" % len(gal_zD))

pcm=ax4.pcolormesh(X, Y, XCorrPlot_3d,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax4.set_aspect('equal')
ax4.set_xlim(np.min(X), SigMax)
ax4.set_ylim(PiMin, PiMax)
ax4.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax4.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax4.set_title("3DHST: %i galaxies" % len(gal_3d))

pcm=ax5.pcolormesh(X, Y, XCorrPlot_clamato,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax5.set_aspect('equal')
ax5.set_xlim(np.min(X), SigMax)
ax5.set_ylim(PiMin, PiMax)
ax5.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax5.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax5.set_title("CLAMATO: %i galaxies" % len(gal_clamato))


ax6pos = ax6.get_position()
cbaxes = fig.add_axes([ax6pos.x0+0.04, ax6pos.y0+0.25, ax6pos.width, ax6pos.height*0.8])
cbaxes.axis('off')
fig.colorbar(pcm, ax=cbaxes)

#fig.subplots_adjust(wspace=-0.1)

plt.show()

In [ ]:
def average_xcorrs(survey_name):
    xcorr_path_list = glob.glob(os.path.join(MOCK_XCORR_DIR, f"xcorrmock_{survey_name}_*_{constants.DATA_VERSION}.npy"))
    xcorr_plot_sum = np.zeros_like(XCorrPlot_3d)
    for i, p in enumerate(xcorr_path_list):
        xcorr_plot_sum += np.load(p).T
    return xcorr_plot_sum / len(xcorr_path_list)

Avg_XCorrPlot_mosdef = average_xcorrs('mosdef')
Avg_XCorrPlot_vuds = average_xcorrs('vuds')
Avg_XCorrPlot_zD = average_xcorrs('zDeep')
Avg_XCorrPlot_3d = average_xcorrs('3dhst')
Avg_XCorrPlot_clamato = average_xcorrs('clamato')

np.save(f'avg_xcorrmock_mosdef_{constants.DATA_VERSION}.npy', Avg_XCorrPlot_mosdef.T)
np.save(f'avg_xcorrmock_vuds_{constants.DATA_VERSION}.npy', Avg_XCorrPlot_vuds.T)
np.save(f'avg_xcorrmock_zDeep_{constants.DATA_VERSION}.npy', Avg_XCorrPlot_zD.T)
np.save(f'avg_xcorrmock_3dhst_{constants.DATA_VERSION}.npy', Avg_XCorrPlot_3d.T)
np.save(f'avg_xcorrmock_clamato_{constants.DATA_VERSION}.npy', Avg_XCorrPlot_clamato.T)

In [ ]:
# TODO: comment this out!
# For trying to get a better guess at sigma-z, divide the x-corr by some fudge factor until the background-to-peak matches the observations approximately.
# Then displace the background for the mock x-corr.
# f = lambda xcorr: (xcorr * (1.1 / 1.4)) - 0.015

# Avg_XCorrPlot_mosdef = f(Avg_XCorrPlot_mosdef)
# Avg_XCorrPlot_vuds = f(Avg_XCorrPlot_vuds)
# Avg_XCorrPlot_zD = f(Avg_XCorrPlot_zD)
# Avg_XCorrPlot_3d = f(Avg_XCorrPlot_3d)
# Avg_XCorrPlot_clamato = f(Avg_XCorrPlot_clamato)

In [ ]:
X, Y = np.meshgrid(SigEdges, PiEdges)

SigMax = 35.
PiMin = -40.
PiMax = 40.

fig, ((ax1, ax2, ax3),(ax4,ax5,ax6))= plt.subplots(2,3, figsize=(12,14))

print(len(X), len(Y), np.shape(XCorrPlot_mosdef))

ax1.pcolormesh(X, Y, Avg_XCorrPlot_mosdef,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax1.set_aspect('equal')
ax1.set_xlim(np.min(X), SigMax)
ax1.set_ylim(PiMin, PiMax)
ax1.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax1.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax1.set_title("MOSDEF: %i galaxies" % len(gal_mosdef))

ax2.pcolormesh(X, Y, Avg_XCorrPlot_vuds,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax2.set_aspect('equal')
ax2.set_xlim(np.min(X), SigMax)
ax2.set_ylim(PiMin, PiMax)
ax2.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax2.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax2.set_title("VUDS: %i galaxies" % len(gal_vuds))

ax3.pcolormesh(X, Y, Avg_XCorrPlot_zD,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax3.set_aspect('equal')
ax3.set_xlim(np.min(X), SigMax)
ax3.set_ylim(PiMin, PiMax)
ax3.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax3.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax3.set_title("zCOSMOS: %i galaxies" % len(gal_zD))

pcm=ax4.pcolormesh(X, Y, Avg_XCorrPlot_3d,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax4.set_aspect('equal')
ax4.set_xlim(np.min(X), SigMax)
ax4.set_ylim(PiMin, PiMax)
ax4.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax4.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax4.set_title("3DHST: %i galaxies" % len(gal_3d))

pcm=ax5.pcolormesh(X, Y, Avg_XCorrPlot_clamato,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax5.set_aspect('equal')
ax5.set_xlim(np.min(X), SigMax)
ax5.set_ylim(PiMin, PiMax)
ax5.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax5.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax5.set_title("CLAMATO: %i galaxies" % len(gal_clamato))


ax6pos = ax6.get_position()
cbaxes = fig.add_axes([ax6pos.x0+0.04, ax6pos.y0+0.25, ax6pos.width, ax6pos.height*0.8])
cbaxes.axis('off')
fig.colorbar(pcm, ax=cbaxes)

#fig.subplots_adjust(wspace=-0.1)

plt.savefig('figures/mock-avg-xcorr-2d.png')
plt.show()

In [ ]:
pi_flat_plot = lambda xcorr, lab: plt.plot(PiEdges[1:], np.sum(xcorr, axis=1), label=lab)

pi_flat_plot(Avg_XCorrPlot_mosdef, 'MOSDEF')
pi_flat_plot(Avg_XCorrPlot_vuds, 'VUDS')
pi_flat_plot(Avg_XCorrPlot_zD, 'zCOSMOS')
pi_flat_plot(Avg_XCorrPlot_3d, '3DHST')
pi_flat_plot(Avg_XCorrPlot_clamato, 'CLAMATO')
plt.xlabel(r'$\pi\; (\mathrm{cMpc})$')
# plt.ylim(-1.5, 0.3)
plt.legend()

In [ ]:
sigma_flat_plot = lambda xcorr, lab: plt.plot(SigEdges[1:], np.sum(xcorr, axis=0), label=lab)

sigma_flat_plot(Avg_XCorrPlot_mosdef, 'MOSDEF')
sigma_flat_plot(Avg_XCorrPlot_vuds, 'VUDS')
sigma_flat_plot(Avg_XCorrPlot_zD, 'zCOSMOS')
sigma_flat_plot(Avg_XCorrPlot_3d, '3DHST')
sigma_flat_plot(Avg_XCorrPlot_clamato, 'CLAMATO')
plt.xlabel(r'$\sigma\; (\mathrm{cMpc})$')
plt.xlim(0, 35)
# plt.ylim(-5, 0.5)
plt.legend()
plt.savefig('figures/mock-avg-xcorr-sigma.png')